<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/09_%E3%83%8F%E3%83%9F%E3%83%AB%E3%83%88%E3%83%B3%E3%83%91%E3%82%B9%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### ハミルトンパス問題 ####################

#すべての辺を有向辺として入力  （無向グラフを考える場合は双方向を入力）
N,M=map(int,input().split()) #頂点数、辺の数
edge=set()  #辺集合
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.add((u,v))

#バイナリ変数の生成
gen=VariableGenerator()
q=gen.array("Binary",shape=(N,N))  #頂点vがサイクルのj番目に訪問されるかどうか

#制約項①　各頂点は1度だけ訪問
f1=0
for v in range(N):
  cnt=0
  for j in range(N):
    cnt+=q[v][j]
  f1+=(1-cnt)**2

#制約項②　1回の移動で訪問するのは1つの頂点のみ
f2=0
for j in range(N):
  cnt=0
  for v in range(N):
    cnt+=q[v][j]
  f2+=(1-cnt)**2

#制約項③　サイクルではグラフに存在する辺のみを通れる
f3=0
for u in range(N):
  for v in range(N):
    if (u,v) in edge:
      continue
    for j in range(N-1):
        f3+=q[u][j]*q[v][j+1]

#ペナルティ係数を設定
A=int(input())
f=A*(f1+f2+f3)


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=list(result.best.values.values())


if objective>0:
  print("ハミルトンサイクルは存在しない")
else:
  cycle=[None]*N
  for pos in range(N*N):
    if values[pos]==1:
      cycle[pos%N]=pos//N+1
  print(f"ハミルトンサイクル：{cycle}")